# SAP Report Cleaner

Dieses Notebook bereinigt SAP-Reports, die als pseudo-XLS-Dateien (eigentlich tab-getrennte Texte) vorliegen.

**Lokale Ausführung:** Dieses Notebook läuft lokal mit Python, ohne Google Colab.

## Funktionen:
- Entfernt Datumsinformationen aus Zeile 1
- Filtert nur Spalten C bis Q
- Entfernt Summenzeilen (markiert mit "*" in Spalte B)
- Entfernt leere Zeilen
- Entfernt Zeilen ohne Materialnummer in Spalte C
- Bereinigt Zahlenformate für pandas
- Konvertiert Datumsformate (DD.MM.YY)
- Exportiert als saubere CSV-Datei und Excel-Datei
- Speichert gelöschte Zeilen zur Nachverfolgung

## Voraussetzungen:
```bash
pip install pandas openpyxl
```

## Verwendung:
1. Alle Zellen des Notebooks ausführen
2. Bei der Dateiauswahl haben Sie drei Optionen:
   - **Option 1:** Dateipfad direkt in Zelle 4 angeben (für wiederholte Nutzung)
   - **Option 2:** Datei-Dialog verwenden (falls tkinter verfügbar)
   - **Option 3:** Dateipfad interaktiv eingeben
3. Die bereinigten Dateien werden im gleichen Verzeichnis wie die Eingabedatei gespeichert


In [8]:
# Installation und Imports
import pandas as pd
import numpy as np
from datetime import datetime
import re
import os
from pathlib import Path
import sys
import subprocess

# Optional: tkinter für Dateiauswahl-Dialog (falls verfügbar)
try:
    from tkinter import filedialog
    import tkinter as tk
    HAS_TKINTER = True
except ImportError:
    HAS_TKINTER = False
    print("Hinweis: tkinter nicht verfügbar. Dateipfad muss manuell eingegeben werden.")

# Hilfsfunktion: Prüft und importiert fehlende Module
def ensure_imports():
    """Stellt sicher, dass alle benötigten Module importiert sind"""
    import sys
    import importlib
    
    modules = {
        'pd': 'pandas',
        'np': 'numpy',
        'os': 'os',
        'Path': 'pathlib',
        'sys': 'sys',
        'subprocess': 'subprocess'
    }
    
    for var_name, module_name in modules.items():
        if var_name not in globals():
            if var_name == 'Path':
                from pathlib import Path
                globals()['Path'] = Path
            elif var_name == 'pd':
                import pandas as pd
                globals()['pd'] = pd
            elif var_name == 'np':
                import numpy as np
                globals()['np'] = np
            else:
                module = importlib.import_module(module_name)
                globals()[var_name] = module

# Stelle sicher, dass alle Imports verfügbar sind
ensure_imports()

# Einfacher Import-Fallback für andere Zellen (kann in jeder Zelle verwendet werden)
def _import_pandas():
    """Importiert pandas falls nicht verfügbar - kann in jeder Zelle aufgerufen werden"""
    try:
        return pd
    except NameError:
        import pandas as pd
        return pd


In [5]:
# Hilfsfunktion zum Installieren von Paketen (versucht verschiedene Methoden)
def install_package(package_name):
    """Versucht ein Paket mit verschiedenen pip-Methoden zu installieren"""
    methods = [
        # Methode 1: pip3 direkt
        ["pip3", "install", package_name, "-q"],
        # Methode 2: python3 -m pip
        ["python3", "-m", "pip", "install", package_name, "-q"],
        # Methode 3: sys.executable -m pip (aktueller Interpreter)
        [sys.executable, "-m", "pip", "install", package_name, "-q"],
        # Methode 4: pip (falls verfügbar)
        ["pip", "install", package_name, "-q"],
    ]
    
    for method in methods:
        try:
            subprocess.check_call(method, 
                                stdout=subprocess.DEVNULL, 
                                stderr=subprocess.DEVNULL)
            return True
        except (subprocess.CalledProcessError, FileNotFoundError):
            continue
    return False

# openpyxl für Excel-Export prüfen und ggf. installieren
try:
    import openpyxl
    print("✓ openpyxl verfügbar")
except ImportError:
    print("⚠ openpyxl nicht gefunden. Installiere automatisch...")
    if install_package("openpyxl"):
        try:
            import openpyxl
            print("✓ openpyxl erfolgreich installiert")
        except ImportError:
            print("⚠ Installation erfolgreich, aber Import fehlgeschlagen. Bitte Kernel neu starten.")
    else:
        print("⚠ Automatische Installation fehlgeschlagen.")
        print("Bitte manuell installieren mit: pip3 install openpyxl")


⚠ openpyxl nicht gefunden. Installiere automatisch...
✓ openpyxl erfolgreich installiert


## 1. Datei auswählen


In [2]:
# Datei auswählen
# Sicherstellen, dass benötigte Imports verfügbar sind (falls Zelle 1 nicht ausgeführt wurde)
try:
    Path
except NameError:
    from pathlib import Path
    import os

# Option 1: Dateipfad direkt hier angeben (für wiederholte Nutzung)
# Beispiele:
# DATEIPFAD_DIRECT = "../sourceDateien/L91_MATverbrauch.xls"  # Relativ vom Notebook-Verzeichnis
# DATEIPFAD_DIRECT = "sourceDateien/L91_MATverbrauch.xls"  # Relativ vom Arbeitsverzeichnis
# DATEIPFAD_DIRECT = "/Users/username/pfad/zur/datei.xls"  # Absoluter Pfad
DATEIPFAD_DIRECT = "../sourceDateien/L91_MATverbrauch.xls"  # Entkommentieren und anpassen
#DATEIPFAD_DIRECT = None  # Auf None setzen für interaktive Auswahl

file_name = None

# Option 1: Dateipfad aus Code verwenden
if DATEIPFAD_DIRECT:
    # Notebook-Verzeichnis finden (wo das Notebook liegt)
    # In Jupyter Notebooks ist das Arbeitsverzeichnis normalerweise das Notebook-Verzeichnis
    current_dir = Path.cwd()
    
    # Prüfe ob wir im Notebooks-Verzeichnis sind
    if current_dir.name == "Notebooks":
        notebook_dir = current_dir
    else:
        # Versuche Notebooks-Verzeichnis relativ zum aktuellen Verzeichnis zu finden
        notebooks_dir = current_dir / "Notebooks"
        if notebooks_dir.exists():
            notebook_dir = notebooks_dir
        else:
            # Fallback: Verwende aktuelles Verzeichnis
            notebook_dir = current_dir
    
    print(f"📁 Notebook-Verzeichnis: {notebook_dir}")
    print(f"📁 Aktuelles Arbeitsverzeichnis: {Path.cwd()}")
    
    # Pfad auflösen
    if os.path.isabs(DATEIPFAD_DIRECT):
        # Absoluter Pfad
        file_name = DATEIPFAD_DIRECT
    else:
        # Relativer Pfad - versuche verschiedene Möglichkeiten
        # 1. Relativ zum Notebook-Verzeichnis
        potential_path = notebook_dir / DATEIPFAD_DIRECT
        if potential_path.exists():
            file_name = str(potential_path.resolve())
        else:
            # 2. Relativ zum aktuellen Arbeitsverzeichnis
            potential_path = Path.cwd() / DATEIPFAD_DIRECT
            if potential_path.exists():
                file_name = str(potential_path.resolve())
            else:
                # 3. Als absoluter Pfad (falls mit / beginnt, aber nicht wirklich absolut)
                file_name = os.path.abspath(DATEIPFAD_DIRECT)
    
    if file_name and os.path.exists(file_name):
        print(f"✓ Dateipfad aus Code verwendet: {file_name}")
    else:
        print(f"⚠ Dateipfad aus Code nicht gefunden: {DATEIPFAD_DIRECT}")
        print(f"   Versuchte Pfade:")
        print(f"   - {notebook_dir / DATEIPFAD_DIRECT}")
        print(f"   - {Path.cwd() / DATEIPFAD_DIRECT}")
        print(f"   - {os.path.abspath(DATEIPFAD_DIRECT)}")
        file_name = None

# Option 2: Datei-Dialog verwenden (falls tkinter verfügbar und noch keine Datei)
if not file_name and HAS_TKINTER:
    root = tk.Tk()
    root.withdraw()  # Hauptfenster verstecken
    file_path = filedialog.askopenfilename(
        title="SAP Report Datei auswählen",
        filetypes=[("XLS Dateien", "*.xls"), ("Alle Dateien", "*.*")]
    )
    root.destroy()
    
    if file_path:
        file_name = file_path
        print(f"✓ Ausgewählte Datei: {file_name}")

# Option 3: Dateipfad manuell eingeben (falls noch keine Datei)
if not file_name:
    print("Bitte geben Sie den vollständigen Pfad zur SAP Report Datei ein:")
    print("Beispiel: /Users/username/pfad/zur/datei.xls")
    print("Oder relativ zum aktuellen Verzeichnis: sourceDateien/L91_MATverbrauch.xls")
    input_path = input("Dateipfad: ").strip().strip('"').strip("'")
    
    # Relativen Pfad zu absolutem Pfad konvertieren
    if not os.path.isabs(input_path):
        file_name = os.path.abspath(input_path)
    else:
        file_name = input_path

# Dateipfad validieren
if not file_name or not os.path.exists(file_name):
    raise FileNotFoundError(f"Datei nicht gefunden: {file_name}")

print(f"✓ Datei gefunden: {file_name}")
print(f"  Vollständiger Pfad: {os.path.abspath(file_name)}")


📁 Notebook-Verzeichnis: /Users/kuehnej/dev/Colab_Skripte_OEE/Notebooks
📁 Aktuelles Arbeitsverzeichnis: /Users/kuehnej/dev/Colab_Skripte_OEE/Notebooks
✓ Dateipfad aus Code verwendet: /Users/kuehnej/dev/Colab_Skripte_OEE/sourceDateien/L91_MATverbrauch.xls
✓ Datei gefunden: /Users/kuehnej/dev/Colab_Skripte_OEE/sourceDateien/L91_MATverbrauch.xls
  Vollständiger Pfad: /Users/kuehnej/dev/Colab_Skripte_OEE/sourceDateien/L91_MATverbrauch.xls


In [ ]:
# DATEI-ANALYSE: Untersuche die Dateistruktur genau
print("=" * 120)
print("DATEI-ANALYSE: L91_MATverbrauch.xls")
print("=" * 120)

# Datei roh einlesen
with open(file_name, 'r', encoding='utf-8', errors='ignore') as f:
    raw_lines = f.readlines()

print(f"\nGesamtanzahl Zeilen: {len(raw_lines)}")
print(f"Erste 10 Zeilen analysieren:\n")

for i in range(min(10, len(raw_lines))):
    line = raw_lines[i]
    line_num = i + 1
    
    print(f"{'='*120}")
    print(f"ZEILE {line_num}:")
    print(f"  Länge (Zeichen): {len(line)}")
    print(f"  Länge (ohne \\n): {len(line.rstrip())}")
    print(f"  Enthält Tabs: {chr(9) in line}")
    print(f"  Anzahl Tabs: {line.count(chr(9))}")
    
    # Zeige rohen Inhalt (repr für sichtbare Zeichen)
    print(f"  Roher Inhalt (repr, erste 200 Zeichen):")
    print(f"    {repr(line[:200])}")
    
    # Split bei Tabs
    parts = line.rstrip('\n\r').split('\t')
    print(f"  Anzahl Teile nach Tab-Split: {len(parts)}")
    print(f"  Erste 10 Teile:")
    for idx, part in enumerate(parts[:10]):
        if part.strip():
            print(f"    [{idx}] (Länge {len(part)}): '{part[:50]}'")
        else:
            print(f"    [{idx}] (leer)")
    
    # Speziell für Zeile 4 (Header)
    if line_num == 4:
        print(f"\n  → ZEILE 4 (Header-Zeile) - Detaillierte Analyse:")
        print(f"    Spalte A (Index 0): '{parts[0] if len(parts) > 0 else 'N/A'}'")
        print(f"    Spalte B (Index 1): '{parts[1] if len(parts) > 1 else 'N/A'}'")
        print(f"    Spalte C (Index 2): '{parts[2] if len(parts) > 2 else 'N/A'}'")
        print(f"    Spalte D (Index 3): '{parts[3] if len(parts) > 3 else 'N/A'}'")
        print(f"    Alle Spalten C-Q (Index 2-16):")
        for idx in range(2, min(17, len(parts))):
            col_letter = chr(ord('A') + idx)
            print(f"      {col_letter} (Index {idx}): '{parts[idx].strip()}'")
    
    print()

print("=" * 120)


## 2. Datei einlesen und vorbereiten


In [3]:
# Datei als tab-getrennten Text einlesen
with open(file_name, 'r', encoding='utf-8', errors='ignore') as f:
    lines = f.readlines()

print(f"Gesamtanzahl Zeilen: {len(lines)}")
print(f"Erste 5 Zeilen:")
for i, line in enumerate(lines[:5], 1):
    print(f"{i}: {repr(line[:100])}")


Gesamtanzahl Zeilen: 903
Erste 5 Zeilen:
1: '0\x008\x00.\x000\x001\x00.\x002\x000\x002\x006\x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00'
2: '\x00\n'
3: '\x00\n'
4: '\x00\n'
5: '\x00\n'


## 3. Daten bereinigen


In [14]:
# DEBUG: Zeige die ersten 100 Spalten und 100 Zeilen
print("=" * 120)
print("DEBUG: Erste 100 Spalten und 100 Zeilen")
print("=" * 120)

# Spaltenbuchstaben für Anzeige (A-Z, dann AA-AZ, etc.)
def get_col_letter(col_idx):
    """Konvertiert Spaltenindex zu Spaltenbuchstaben (A, B, ..., Z, AA, AB, ...)"""
    result = ""
    col_idx += 1  # 0-basiert zu 1-basiert
    while col_idx > 0:
        col_idx -= 1
        result = chr(ord('A') + (col_idx % 26)) + result
        col_idx //= 26
    return result

max_rows = min(100, len(data_rows))
max_cols = 100

print(f"\nZeige {max_rows} Zeilen und {max_cols} Spalten\n")

for i in range(max_rows):
    row = data_rows[i]['row']
    original_idx = data_rows[i]['original_index']
    
    print(f"\n{'='*120}")
    print(f"ZEILE {original_idx:3d} (Row-Index {i:3d}) - Anzahl Spalten: {len(row)}")
    print(f"{'='*120}")
    
    # Zeige alle Spalten in kompakter Form (mehrere Spalten pro Zeile)
    cols_per_line = 10  # 10 Spalten pro Zeile für bessere Lesbarkeit
    
    for start_col in range(0, min(max_cols, len(row)), cols_per_line):
        end_col = min(start_col + cols_per_line, max_cols, len(row))
        
        # Header-Zeile mit Spaltenbuchstaben
        header_line = "  "
        value_line = "  "
        
        for col_idx in range(start_col, end_col):
            col_letter = get_col_letter(col_idx)
            if col_idx < len(row):
                cell_value = row[col_idx].strip() if row[col_idx] else ''
                if not cell_value:
                    cell_value = ''
                # Kürze lange Werte
                if len(cell_value) > 30:
                    cell_value = cell_value[:27] + "..."
            else:
                cell_value = ''
            
            # Formatierung
            header_line += f"{col_letter:>8} "
            value_line += f"{cell_value[:8]:>8} "
        
        print(header_line)
        print(value_line)
        print()  # Leerzeile zwischen Gruppen
    
    # Wenn Zeile weniger Spalten hat als max_cols
    if len(row) < max_cols:
        print(f"  ... (nur {len(row)} Spalten vorhanden)")
    
    # Speziell für Zeile 4: Zeige auch Spalten C-Q im Detail
    if original_idx == 4:
        print(f"\n{'='*120}")
        print("→ ZEILE 4 (Header-Zeile) - Spalten C-Q im Detail:")
        print(f"{'='*120}")
        cols_c_to_q = []
        for j in range(2, min(17, len(row) + 1)):
            if j < len(row):
                cell = row[j].strip() if row[j] else ''
                cols_c_to_q.append(cell)
            else:
                cols_c_to_q.append('')
        
        for idx, header in enumerate(cols_c_to_q, start=0):
            col_letter = chr(ord('C') + idx) if idx < 15 else 'Q'
            print(f"  Spalte {col_letter} (Index {idx+2:2d}): '{header}'")

print("\n" + "=" * 120)
print(f"Debug abgeschlossen: {max_rows} Zeilen, bis zu {max_cols} Spalten pro Zeile")
print("=" * 120)


DEBUG: Erste 100 Spalten und 100 Zeilen

Zeige 100 Zeilen und 100 Spalten


ZEILE   2 (Row-Index   0) - Anzahl Spalten: 17
         A        B        C        D        E        F        G        H        I        J 
                                                                                            

         K        L        M        N        O        P        Q 
                                                                 

  ... (nur 17 Spalten vorhanden)

ZEILE   3 (Row-Index   1) - Anzahl Spalten: 17
         A        B        C        D        E        F        G        H        I        J 
                                                                                            

         K        L        M        N        O        P        Q 
                                                                 

  ... (nur 17 Spalten vorhanden)

ZEILE   4 (Row-Index   2) - Anzahl Spalten: 17
         A        B        C        D        E        F        G        H  

In [12]:
# VERBESSERTE Header-Erkennung - ersetzt die alte Logik
# Diese Zelle überschreibt die Header-Erkennung mit einer robusteren Version

print("\n" + "=" * 80)
print("VERBESSERTE Header-Erkennung")
print("=" * 80)

header_row_idx = None
actual_headers = None

for i, data_row in enumerate(data_rows):
    row = data_row['row']
    original_idx = data_row['original_index']
    
    # Prüfen ob diese Zeile die Header enthält (mindestens Spalte C benötigt)
    if len(row) >= 3:
        # Spalte C (Index 2) extrahieren
        col_c = row[2].strip() if len(row) > 2 and row[2] else ''
        
        # Prüfe ob "Material" in Spalte C steht (robust: ignoriere Leerzeichen und Groß-/Kleinschreibung)
        col_c_normalized = ''.join(col_c.lower().split())
        material_normalized = ''.join('Material'.lower().split())
        
        if col_c_normalized == material_normalized or 'material' in col_c_normalized:
            # Spalten C bis Q (Index 2-16) extrahieren
            potential_headers = []
            for j in range(2, min(17, len(row) + 1)):
                if j < len(row):
                    cell = row[j].strip() if row[j] else ''
                    potential_headers.append(cell)
                else:
                    potential_headers.append('')
            
            header_row_idx = i
            actual_headers = potential_headers
            print(f"✓ Header-Zeile gefunden in Zeile {original_idx} (Row-Index {i})")
            print(f"  Spalte C enthält: '{col_c}'")
            print(f"  Anzahl gefundener Header: {len(potential_headers)}")
            print(f"  Erste 5 Header: {potential_headers[:5]}")
            break

# Überschreibe die alte Variable falls gefunden
if header_row_idx is not None:
    # Entferne die Header-Zeile aus data_rows (falls noch vorhanden)
    if header_row_idx < len(data_rows):
        header_data = data_rows[header_row_idx]
        # Prüfe ob diese Zeile noch nicht gelöscht wurde
        already_deleted = any(d.get('Zeile') == header_data['original_index'] and d.get('Grund') == 'Header-Zeile' 
                             for d in deleted_rows)
        if not already_deleted:
            data_rows.pop(header_row_idx)
            deleted_rows.append({
                'Zeile': header_data['original_index'],
                'Grund': 'Header-Zeile',
                'Inhalt': '\t'.join(header_data['row'])[:100]
            })
    print(f"✓ Header-Zeile erfolgreich identifiziert und entfernt")
    print("=" * 80)
elif 'header_row_idx' in globals() and header_row_idx is None:
    print("⚠ Header-Zeile nicht gefunden - verwende Fallback")
    actual_headers = expected_headers
    if len(data_rows) > 3:
        header_row_idx = 3
    elif len(data_rows) > 0:
        header_row_idx = 0
    print("=" * 80)



VERBESSERTE Header-Erkennung
⚠ Header-Zeile nicht gefunden - verwende Fallback


In [9]:
# VERBESSERTE Header-Erkennung (ersetzt die alte Logik)
# Diese Zelle enthält eine robustere Header-Erkennung, die Leerzeichen ignoriert
# und nach mehreren erwarteten Headern sucht

# Hilfsfunktion: Normalisiert Text für Vergleich
def normalize_header(text):
    """Normalisiert Header-Text: entfernt Leerzeichen, macht lowercase"""
    if not text:
        return ''
    return ''.join(str(text).lower().split())

# Durchsuche alle Zeilen nach Header
header_row_idx = None
actual_headers = None

print("Suche nach Header-Zeile (robuste Erkennung)...")
for i, data_row in enumerate(data_rows):
    row = data_row['row']
    if len(row) < 3:  # Mindestens 3 Spalten benötigt
        continue
    
    # Spalten C bis Q extrahieren (Index 2-16)
    potential_headers = []
    for j in range(2, min(17, len(row) + 1)):
        if j < len(row):
            cell = row[j].strip() if row[j] else ''
            potential_headers.append(cell)
        else:
            potential_headers.append('')
    
    # Normalisierte Versionen für Vergleich
    normalized_potential = [normalize_header(h) for h in potential_headers]
    normalized_expected = [normalize_header(h) for h in expected_headers]
    
    # Prüfe auf mehrere Übereinstimmungen (mindestens 3 Header müssen übereinstimmen)
    matches = 0
    for idx, exp_header in enumerate(normalized_expected):
        if idx < len(normalized_potential):
            potential = normalized_potential[idx]
            # Exakte Übereinstimmung
            if potential == exp_header:
                matches += 1
            # Teilübereinstimmung
            elif exp_header and potential:
                if exp_header in potential or potential in exp_header:
                    matches += 1
                # Prüfe ob wichtige Wörter übereinstimmen
                elif any(word in potential for word in exp_header.split() if len(word) > 2):
                    matches += 1
    
    # Wenn mindestens 3 Header übereinstimmen, ist es wahrscheinlich die Header-Zeile
    if matches >= 3:
        header_row_idx = i
        actual_headers = potential_headers
        print(f"✓ Header-Zeile gefunden in Zeile {data_row['original_index']} ({matches} Übereinstimmungen)")
        print(f"  Gefundene Header (erste 5): {actual_headers[:5]}")
        break

# Überschreibe die alte header_row_idx Variable falls gefunden
if header_row_idx is not None:
    # Entferne die alte Header-Zeile aus data_rows (falls noch vorhanden)
    if header_row_idx < len(data_rows):
        header_data = data_rows[header_row_idx]
        # Prüfe ob diese Zeile noch nicht gelöscht wurde
        if any(d['Zeile'] == header_data['original_index'] for d in deleted_rows if d.get('Grund') == 'Header-Zeile'):
            pass  # Bereits gelöscht
        else:
            data_rows.pop(header_row_idx)
            deleted_rows.append({
                'Zeile': header_data['original_index'],
                'Grund': 'Header-Zeile',
                'Inhalt': '\t'.join(header_data['row'])[:100]
            })
    print(f"✓ Header-Zeile erfolgreich identifiziert und entfernt")
elif 'header_row_idx' in globals() and header_row_idx is None:
    print("⚠ Warnung: Header-Zeile nicht gefunden mit verbesserter Erkennung")
    print("  Verwende Fallback auf erwartete Header")
    actual_headers = expected_headers
    if len(data_rows) > 3:
        header_row_idx = 3
    elif len(data_rows) > 0:
        header_row_idx = 0


Suche nach Header-Zeile (robuste Erkennung)...
⚠ Warnung: Header-Zeile nicht gefunden mit verbesserter Erkennung
  Verwende Fallback auf erwartete Header


In [4]:
# Zeilen in Spalten aufteilen (tab-getrennt)
data_rows = []
deleted_rows = []  # Für Nachverfolgung gelöschter Zeilen

for idx, line in enumerate(lines, start=1):
    # Tab-getrennte Werte extrahieren
    row = line.rstrip('\n\r').split('\t')
    
    # Zeile 1 überspringen (Datum)
    if idx == 1:
        deleted_rows.append({
            'Zeile': idx,
            'Grund': 'Datum in Zeile 1',
            'Inhalt': line[:100]
        })
        continue
    
    # Leere Zeilen überspringen
    if not line.strip():
        deleted_rows.append({
            'Zeile': idx,
            'Grund': 'Leere Zeile',
            'Inhalt': ''
        })
        continue
    
    # Mindestens 17 Spalten benötigt (A=0 bis Q=16)
    if len(row) < 17:
        # Prüfen ob es eine komplett leere Zeile ist
        if all(not cell.strip() for cell in row):
            deleted_rows.append({
                'Zeile': idx,
                'Grund': 'Komplett leere Zeile',
                'Inhalt': line[:100]
            })
            continue
        else:
            # Zeile mit weniger Spalten - mit leeren Werten auffüllen
            row.extend([''] * (17 - len(row)))
    
    data_rows.append({
        'original_index': idx,
        'row': row
    })

print(f"Verarbeitete Zeilen: {len(data_rows)}")
print(f"Gelöschte Zeilen: {len(deleted_rows)}")


Verarbeitete Zeilen: 902
Gelöschte Zeilen: 1


In [5]:
# Spaltenüberschriften finden (sollten in Zeile 4 stehen)
# Erwartete Spalten: Material, Functional Loc., Equipment, Material Description, Work Ctr, 
# Withdrawn, W/o resrv., Reserved, Reserv.ref, Pstng Date, Order, ID, Message, ICt, Customer

expected_headers = [
    'Material', 'Functional Loc.', 'Equipment', 'Material Description', 'Work Ctr',
    'Withdrawn', 'W/o resrv.', 'Reserved', 'Reserv.ref', 'Pstng Date',
    'Order', 'ID', 'Message', 'ICt', 'Customer'
]

# Header-Zeile finden (normalerweise Zeile 4, Index 3 in 0-basiert)
header_row_idx = None
for i, data_row in enumerate(data_rows):
    row = data_row['row']
    # Prüfen ob diese Zeile die Header enthält
    if len(row) >= 17:
        # Spalten C bis Q (Index 2-16) extrahieren
        potential_headers = [row[j].strip() if j < len(row) else '' for j in range(2, 17)]
        # Prüfen ob "Material" in Spalte C steht
        if potential_headers[0] == 'Material' or 'Material' in potential_headers[0]:
            header_row_idx = i
            actual_headers = potential_headers
            break

if header_row_idx is None:
    # Fallback: Verwende erwartete Header
    print("Warnung: Header-Zeile nicht gefunden, verwende erwartete Header")
    actual_headers = expected_headers
    header_row_idx = 3  # Standard: Zeile 4
else:
    print(f"Header-Zeile gefunden in Zeile {data_rows[header_row_idx]['original_index']}")
    print(f"Gefundene Header: {actual_headers}")

# Header-Zeile aus data_rows entfernen
if header_row_idx is not None:
    header_data = data_rows.pop(header_row_idx)
    deleted_rows.append({
        'Zeile': header_data['original_index'],
        'Grund': 'Header-Zeile',
        'Inhalt': '\t'.join(header_data['row'])[:100]
    })


Warnung: Header-Zeile nicht gefunden, verwende erwartete Header


In [6]:
# Daten filtern und bereinigen
cleaned_data = []
filtered_out = []

for data_row in data_rows:
    original_idx = data_row['original_index']
    row = data_row['row']
    
    # Spalten C bis Q extrahieren (Index 2-16)
    if len(row) < 17:
        row.extend([''] * (17 - len(row)))
    
    cols_c_to_q = [row[i].strip() if i < len(row) else '' for i in range(2, 17)]
    
    # Prüfung 1: Spalte B (Index 1) hat "*" UND Spalte D (Index 3) ist leer -> Summenzeile
    col_b = row[1].strip() if len(row) > 1 else ''
    col_d = row[3].strip() if len(row) > 3 else ''
    
    if col_b == '*' and not col_d:
        filtered_out.append({
            'Zeile': original_idx,
            'Grund': 'Summenzeile (* in Spalte B, leer in Spalte D)',
            'Inhalt': '\t'.join(row)[:100]
        })
        continue
    
    # Prüfung 2: Keine Materialnummer in Spalte C (Index 2, erste Spalte in cols_c_to_q)
    material_num = cols_c_to_q[0] if len(cols_c_to_q) > 0 else ''
    if not material_num or material_num == '':
        filtered_out.append({
            'Zeile': original_idx,
            'Grund': 'Keine Materialnummer in Spalte C',
            'Inhalt': '\t'.join(row)[:100]
        })
        continue
    
    # Prüfung 3: Komplett leere Zeile in Spalten C-Q
    if all(not cell.strip() for cell in cols_c_to_q):
        filtered_out.append({
            'Zeile': original_idx,
            'Grund': 'Komplett leere Zeile in Spalten C-Q',
            'Inhalt': '\t'.join(row)[:100]
        })
        continue
    
    # Zeile behalten
    cleaned_data.append(cols_c_to_q)

print(f"Bereinigte Datenzeilen: {len(cleaned_data)}")
print(f"Gefilterte Zeilen: {len(filtered_out)}")

# Gelöschte Zeilen zur Nachverfolgung hinzufügen
deleted_rows.extend(filtered_out)


Bereinigte Datenzeilen: 343
Gefilterte Zeilen: 558


In [ ]:
# DataFrame erstellen
df = pd.DataFrame(cleaned_data, columns=expected_headers)

print(f"DataFrame Shape: {df.shape}")
print(f"\nErste 5 Zeilen:")
print(df.head())
print(f"\nSpalten: {df.columns.tolist()}")


## 4. Datenformate bereinigen


In [ ]:
# Funktion zum Bereinigen von Zahlen (entfernt Leerzeichen, Tausenderpunkte, etc.)
def clean_number(value):
    if pd.isna(value) or value == '' or str(value).strip() == '':
        return None
    value_str = str(value).strip()
    
    # Alle Leerzeichen entfernen
    value_str = value_str.replace(' ', '')
    
    # Negative Zahlen erkennen (z.B. "-10")
    is_negative = value_str.startswith('-')
    if is_negative:
        value_str = value_str[1:]
    
    # Tausenderpunkte entfernen (z.B. "3.500" -> "3500")
    # SAP verwendet oft Punkte als Tausender-Trennzeichen
    if '.' in value_str:
        # Prüfen ob es ein Tausenderpunkt ist
        # Pattern: Zahl mit Punkt gefolgt von genau 3 Ziffern am Ende = Tausenderpunkt
        # z.B. "3.500" oder "12.345"
        parts = value_str.rsplit('.', 1)  # Split vom Ende
        if len(parts) == 2 and len(parts[1]) == 3 and parts[1].isdigit():
            # Tausenderpunkt entfernen
            value_str = parts[0] + parts[1]
        elif ',' in value_str:
            # Falls Komma vorhanden, könnte es ein Dezimaltrennzeichen sein
            # Aber laut Anforderung sind es Integer, also ignorieren wir Dezimalstellen
            value_str = value_str.replace(',', '')
            value_str = value_str.replace('.', '')
        else:
            # Punkt vorhanden, aber nicht als Tausenderpunkt erkannt -> entfernen
            value_str = value_str.replace('.', '')
    
    # Komma entfernen (falls noch vorhanden)
    value_str = value_str.replace(',', '')
    
    try:
        # Als Integer konvertieren (laut Anforderung sind es Integer)
        result = int(value_str) if value_str else None
        return -result if is_negative and result is not None else result
    except (ValueError, TypeError):
        return None

# Funktion zum Konvertieren von Datum (DD.MM.YY oder DD.MM.YYYY)
def convert_date(value):
    if pd.isna(value) or value == '' or str(value).strip() == '':
        return None
    value_str = str(value).strip()
    try:
        # Format: DD.MM.YY (8 Zeichen)
        if len(value_str) == 8 and value_str.count('.') == 2:
            day, month, year = value_str.split('.')
            # Jahr 00-99 -> 2000-2099
            year_int = int(year)
            if year_int < 50:
                year_int += 2000
            else:
                year_int += 1900
            return f"{day}.{month}.{year_int:04d}"
        # Format: DD.MM.YYYY (10 Zeichen, bereits vollständig)
        elif len(value_str) == 10 and value_str.count('.') == 2:
            return value_str
    except:
        pass
    return value_str  # Falls Konvertierung fehlschlägt, Original zurückgeben


In [ ]:
# Spaltenzuordnung:
# Material (C) bleibt als Text (Identifier)
# D, F, G, L, Q sind Text: Functional Loc., Material Description, Work Ctr, Pstng Date, Customer
# L ist Datum (Index 10) - "Pstng Date" 
# Rest sind Zahlen (Integer)

# Spalten die Text bleiben sollen (inkl. Material als Identifier)
text_columns = ['Material', 'Functional Loc.', 'Equipment', 'Material Description', 'Work Ctr', 'Customer']
# Spalte die Datum ist
date_column = 'Pstng Date'
# Rest sind Zahlen
numeric_columns = [col for col in df.columns if col not in text_columns and col != date_column]

print(f"Text-Spalten: {text_columns}")
print(f"Datum-Spalte: {date_column}")
print(f"Zahlen-Spalten: {numeric_columns}")


In [ ]:
# Datum-Spalte konvertieren
if date_column in df.columns:
    df[date_column] = df[date_column].apply(convert_date)
    print(f"Datum-Spalte '{date_column}' konvertiert")
    print(f"Beispielwerte: {df[date_column].head().tolist()}")

# Zahlen-Spalten bereinigen
for col in numeric_columns:
    if col in df.columns:
        df[col] = df[col].apply(clean_number)
        print(f"Zahlen-Spalte '{col}' bereinigt")

print("\nDataFrame Info:")
print(df.info())
print("\nErste 5 Zeilen nach Bereinigung:")
print(df.head())


## 5. Gelöschte Zeilen zur Nachverfolgung


In [ ]:
# DataFrame für gelöschte Zeilen erstellen
df_deleted = pd.DataFrame(deleted_rows)

print(f"Anzahl gelöschter Zeilen: {len(df_deleted)}")
print(f"\nGelöschte Zeilen nach Grund:")
print(df_deleted['Grund'].value_counts())
print(f"\nErste 10 gelöschte Zeilen:")
print(df_deleted.head(10))


## 6. Export als CSV


In [ ]:
# Output-Dateinamen erstellen (im gleichen Verzeichnis wie Eingabedatei)
input_path = Path(file_name)
output_dir = input_path.parent
base_name = input_path.stem  # Name ohne Erweiterung

output_csv = str(output_dir / f"{base_name}_cleaned.csv")
output_excel = str(output_dir / f"{base_name}_cleaned.xlsx")
output_deleted_csv = str(output_dir / f"{base_name}_deleted_rows.csv")

# Bereinigte Daten als CSV exportieren
df.to_csv(output_csv, index=False, encoding='utf-8-sig', sep=';')
print(f"Bereinigte Daten als CSV gespeichert: {output_csv}")
print(f"Anzahl Zeilen: {len(df)}")
print(f"Anzahl Spalten: {len(df.columns)}")

# Excel-Datei mit mehreren Sheets erstellen (bereinigte Daten + gelöschte Zeilen)
try:
    import openpyxl
except ImportError:
    # Versuche openpyxl nochmal zu installieren (falls beim Start fehlgeschlagen)
    print("\n⚠ openpyxl nicht verfügbar. Versuche Installation...")
    if install_package("openpyxl"):
        try:
            import openpyxl
            print("✓ openpyxl erfolgreich installiert")
        except ImportError:
            print("⚠ Installation erfolgreich, aber Import fehlgeschlagen.")
            openpyxl = None
    else:
        print("⚠ Installation fehlgeschlagen.")
        openpyxl = None

if 'openpyxl' in globals() and openpyxl is not None:
    try:
        with pd.ExcelWriter(output_excel, engine='openpyxl') as writer:
            df.to_excel(writer, sheet_name='Bereinigte Daten', index=False)
            if len(df_deleted) > 0:
                df_deleted.to_excel(writer, sheet_name='Gelöschte Zeilen', index=False)
        print(f"\n✓ Excel-Datei mit mehreren Sheets erstellt: {output_excel}")
        print(f"  - Sheet 'Bereinigte Daten': {len(df)} Zeilen")
        if len(df_deleted) > 0:
            print(f"  - Sheet 'Gelöschte Zeilen': {len(df_deleted)} Zeilen")
    except Exception as e:
        print(f"\n⚠ Fehler beim Excel-Export: {e}")
        # Fallback: Gelöschte Zeilen als separate CSV
        if len(df_deleted) > 0:
            df_deleted.to_csv(output_deleted_csv, index=False, encoding='utf-8-sig', sep=';')
            print(f"✓ Gelöschte Zeilen als CSV gespeichert: {output_deleted_csv}")
            print(f"  Anzahl gelöschter Zeilen: {len(df_deleted)}")
else:
    print("\n⚠ Excel-Export nicht möglich: openpyxl nicht verfügbar")
    print("Bitte manuell installieren mit: pip install openpyxl")
    # Fallback: Gelöschte Zeilen als separate CSV
    if len(df_deleted) > 0:
        df_deleted.to_csv(output_deleted_csv, index=False, encoding='utf-8-sig', sep=';')
        print(f"✓ Gelöschte Zeilen als CSV gespeichert: {output_deleted_csv}")
        print(f"  Anzahl gelöschter Zeilen: {len(df_deleted)}")


In [ ]:
# Dateien lokal speichern
print("\n✅ Export abgeschlossen!")
print(f"📊 Bereinigte Daten (CSV): {output_csv}")
if os.path.exists(output_excel):
    print(f"📊 Excel-Datei: {output_excel}")
if len(df_deleted) > 0 and os.path.exists(output_deleted_csv):
    print(f"🗑️  Gelöschte Zeilen: {output_deleted_csv}")

print(f"\nAlle Dateien wurden gespeichert in: {output_dir}")


## 7. Zusammenfassung


In [ ]:
print("=" * 60)
print("ZUSAMMENFASSUNG")
print("=" * 60)
print(f"Original Datei: {file_name}")
print(f"Gesamtanzahl Zeilen (original): {len(lines)}")
print(f"Bereinigte Datenzeilen: {len(df)}")
print(f"Gelöschte Zeilen: {len(df_deleted)}")
print(f"\nGelöschte Zeilen nach Kategorie:")
for grund, count in df_deleted['Grund'].value_counts().items():
    print(f"  - {grund}: {count}")
print(f"\nOutput-Dateien:")
print(f"  - {output_csv} (CSV)")
try:
    if os.path.exists(output_excel):
        print(f"  - {output_excel} (Excel mit mehreren Sheets)")
except:
    pass
if len(df_deleted) > 0 and not os.path.exists(output_excel):
    print(f"  - {output_deleted_csv} (Gelöschte Zeilen)")
print("=" * 60)
